In [19]:
!pip install keybert

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 86.0/86.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 238.9/238.9 kB 15.2 MB/s eta 0:00:00
     ---------------------------------------- 84.5/84.5 kB ? eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 33.8 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23799 sha256=5a21fc8da545a3b5dc15b9844e8c78e2188c1e725e9b96b24322bfc9aa6815b0
  Stored in directory: c:\users\mahirhiteshshah\appdata\local\pip\cache\wheels\6b\6d\dd\a3e492c7b46f3e3864d7fb4119aa812ebc41ee2808ceb26414
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125940 sha256=3c87d5339f1e19c341436fdc05d0ac1e7e45f05bf17c76fed2defa8b328a69a6
  Stored in 

In [21]:
from transformers import pipeline, AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from keybert import KeyBERT

qa_pipeline = pipeline(
    "question-answering",
    model="mrm8488/spanbert-finetuned-squadv2",
    tokenizer="mrm8488/spanbert-finetuned-squadv2"
)

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
kw_model = KeyBERT()

In [22]:
def similarity(userText, answerText):
    userText = kw_model.extract_keywords(userText, keyphrase_ngram_range=(1,1), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=10)
    userText = ' '.join([x for x, y in userText])
    answerText = kw_model.extract_keywords(answerText, keyphrase_ngram_range=(1,1), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=10)
    answerText = ' '.join([x for x, y in answerText])
    sentences = [userText, answerText]
    tokens = {'input_ids': [], 'attention_mask': []}

    for sentence in sentences:
        new_tokens = tokenizer.encode_plus(sentence, max_length=128, truncation=True,padding='max_length', return_tensors='pt')
        tokens['input_ids'].append(new_tokens['input_ids'][0])
        tokens['attention_mask'].append(new_tokens['attention_mask'][0])

    tokens['input_ids'] = torch.stack(tokens['input_ids'])
    tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
    outputs = model(**tokens)
    embeddings = outputs.last_hidden_state
    attention_mask = tokens['attention_mask']
    mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
    masked_embeddings = embeddings * mask
    summed = torch.sum(masked_embeddings, 1)
    summed_mask = torch.clamp(mask.sum(1), min=1e-9)
    mean_pooled = summed / summed_mask
    mean_pooled = mean_pooled.detach().numpy()
    return cosine_similarity([mean_pooled[0]], mean_pooled[1:])

In [10]:
def answer(passage, question):
    return qa_pipeline({'context': str(passage),'question': str(question)})['answer']